<a href="https://colab.research.google.com/github/derek-shing/DS-Unit-1-Sprint-5-Data-Storytelling-Blog-Post/blob/master/Supreme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Load 1 product info

import requests
import bs4
import pandas as pd
import datetime
import time




## Define function to extract the product information from HTML

In [0]:
def extractinfo(url):
  res = requests.get(url)
  mysoup = bs4.BeautifulSoup(res.text)
  info = mysoup.select('.info')
  type(info[0])

  infohtml = info[0]
  name=infohtml.find('h3').get_text()
  temp=infohtml.find_all('p')
  releasedate=temp[0].get_text()
  price=temp[1].get_text()
  desc=temp[2].get_text()
#infolist=list(infohtml.children)

#infolist.


  releasedate=pd.to_datetime(releasedate, infer_datetime_format=True)
  price=float(price.strip('$').replace(',',''))
  
  info_dict = {'name':name,'price':price,'release_date':releasedate,'desc':desc}
  return info_dict

In [0]:
def get_drop_info(url):
  items=[]
  res=requests.get(url)
  mysoup=bs4.BeautifulSoup(res.text)
  html=mysoup.select('.items a')
  for item in html:
    url='https://supremedatabase.com'+item['href']
    items.append(extractinfo(url))
    
  return items

#url = 'https://supremedatabase.com/seasons/fall-winter-2018/drops/2018-12-06'
#get_drop_info(url)

In [0]:
#Load different day

def get_release_date_info(url):
  items=[]
  #url = 'https://supremedatabase.com/seasons/fall-winter-2018/drops'
  res=requests.get(url)
  mysoup=bs4.BeautifulSoup(res.text)
  html=mysoup.select('.drops p a')
  for item in html:
    url='https://supremedatabase.com'+item['href']
    items.extend(get_drop_info(url))
    
  return items


## Web scrap data from  https://supremedatabase.com/ and put in a Dataframe

In [0]:
product=[]
url='https://supremedatabase.com/seasons/fall-winter-2018/drops'

product.extend(get_release_date_info(url))

In [0]:

url='https://supremedatabase.com/seasons/spring-summer-2018/drops'
product.extend(get_release_date_info(url))

In [10]:
len(product)
#product

405

In [0]:
df = pd.DataFrame(product)

In [12]:
df.sort_values(by='price', ascending=False).head()

,desc,name,price,release_date
240,Lambskin leather with poly and nylon lining. F...,Supreme®/The North Face® Leather Mountain Parka,1098.0,2018-10-19
124,Custom fit Schott® Perfecto leather jacket wit...,Supreme®/Comme des Garçons SHIRT® Schott® Pain...,998.0,2018-09-13
333,Sheepskin with full zip closure and D-ring str...,Patchwork Shearling B-3 Jacket,868.0,2018-11-23
36,Lambskin leather with satin lining and half zi...,Leather Anorak,698.0,2018-08-23
267,Diamond quilted lambskin leather with nickel l...,Quilted Studded Leather Jacket,698.0,2018-11-01


# LOAD DATA FROM https://stockx.com/search?s=supreme

## Set up to use selenium

In [0]:
# install chromium, its driver, and selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..



In [14]:

!pip install selenium
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)

In [0]:
wd = webdriver.Chrome('chromedriver',options=options)

## Get the result-tile tag

In [0]:
def get_result_tile(source):
  xsoup = bs4.BeautifulSoup(source)
  tile_list=xsoup.find(id='root').find(class_='wrapper').find(class_='page-container').find(id='search-wrapper').find(class_='search-results-grid').find_all(class_='result-tile')
  return tile_list

## Get Name and ask price for a tile

In [0]:
def extract_resell_price(tile,key):
  name=tile.find('a').find('h4').get_text()
  price=tile.find('a').find(class_='bid bid-button-b').find('a').find(class_='stats')
  price=price.findChildren("div")[0].get_text()
  if price=='--':
    price=0
  else:
    price=float(price.strip('$').replace(',',''))
  return {'name':name,'resell_price':price,'key':key}
  

## Get result page by searching product name

In [0]:
xlist ={}

#product_name= 'Supreme®/Hanes® Boxer Briefs (4 Pack)'#df.name
#product_name.head()
def get_search_result_page(product_name):
  wd.get('https://stockx.com/search?s=')
  xsoup = bs4.BeautifulSoup(wd.page_source)
  #print(xsoup.find(id='root').find(class_='wrapper'))
  search_bar = wd.find_element_by_id("page-search-bar")
  search_bar
  search_bar.send_keys(product_name)
  time.sleep(1)
  return wd.page_source
#print(wd.page_source)

trial=df.sort_values(by='price', ascending=False)

total = len(trial)
resell_list=[]
loading=0
for product_name in trial.name:
  loading=loading+1
  print('loading', loading,'of',total)
#for url in df.searchurl:
#url=df.searchurl[0]
#print(url)
#result = requests.get(url)
#type(result)
  s=get_search_result_page(product_name)
#print(s)

  tile_list = get_result_tile(s)
  if len(tile_list)>0:
    for tile in tile_list:
      resell_list.append(extract_resell_price(tile,product_name))
  
#resell_list  


#search_box = xsoup.find(id='root').find(class_='wrapper').find(id='site-header').find(class_='page-container vacation-banner-padding').find(id='wrap')

In [0]:
df_x = pd.DataFrame(resell_list)

In [25]:
df_x.head()

,key,name,resell_price
0,Supreme®/The North Face® Leather Mountain Parka,Supreme The North Face Leather Mountain Parka ...,1084.0
1,Supreme®/The North Face® Leather Mountain Parka,Supreme The North Face Leather Mountain Parka ...,1134.0
2,Supreme®/The North Face® Leather Mountain Parka,Supreme The North Face Leather Mountain Parka Red,1200.0
3,Supreme®/The North Face® Leather Mountain Parka,Supreme The North Face Leather Mountain Parka ...,1050.0
4,Supreme®/Comme des Garçons SHIRT® Schott® Pain...,Supreme Comme des Garons SHIRT Schott Painted ...,1249.0


## Saving web scrap result to CSV

In [0]:
from google.colab import files

df_x.to_csv('resell.csv')
files.download('resell.csv')

In [0]:
df.to_csv('product.csv')
files.download('product.csv')

In [0]:
trial.head(10)

## Simulate Scroll down to get more search result html tag ( Not used in this project)

-This is some testing code that may be useful later


In [0]:
from selenium.webdriver.common.keys import Keys
def scrollDown(browser, numberOfScrollDowns):
 body = browser.find_element_by_tag_name("body")
 while numberOfScrollDowns >=0:
  body.send_keys(Keys.PAGE_DOWN)
  numberOfScrollDowns -= 1
 return browser

In [0]:
import time
SCROLL_PAUSE_TIME = 2

# Get scroll height
last_height = wd.execute_script("return document.body.scrollHeight")
last_source=wd.page_source

print('doc height:',last_height)

#for i in range(2):
while True:
    # Scroll down to bottom
    #wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    wd=scrollDown(wd,10)
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_source =wd.page_source
    if new_source==last_source:
        break
    last_source = new_source
    
print(new_source)